# Assignment 2 Theory Naive Bayes

In [ ]:
#Name:Pratiksha Sabale
#PRN: 0120180481
#Roll no: 087


In [ ]:
# Importing library
import math
import random
import pandas as pd
import csv
  
  
# the categorical class names are changed to numberic data

def encode_class(mydata):
    classes = []
    for i in range(len(mydata)):
        if mydata[i][-1] not in classes:
            classes.append(mydata[i][-1])
    for i in range(len(classes)):
        for j in range(len(mydata)):
            if mydata[j][-1] == classes[i]:
                mydata[j][-1] = i
    return mydata            
              
  
# Splitting the data into test and train 
def splitting(mydata, ratio):
    train_num = int(len(mydata) * ratio)
    train = []
    # initally testset will have all the dataset 
    test = list(mydata)
    while len(train) < train_num:
        # index generated randomly from range 0 
        # to length of testset
        index = random.randrange(len(test))
        # from testset, pop data rows and put it in train
        train.append(test.pop(index))
    return train, test
  
  
# Group the data rows under each class yes or 
# no in dictionary eg: dict[yes] and dict[no] 
def groupUnderClass(mydata):
      dict = {}
      for i in range(len(mydata)):
          if (mydata[i][-1] not in dict):
              dict[mydata[i][-1]] = []
          dict[mydata[i][-1]].append(mydata[i])
      return dict
  
  
# Calculating Mean
def mean(numbers):
    return sum(numbers) / float(len(numbers))
  
# Calculating Standard Deviation
def std_dev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x - avg, 2) for x in numbers]) / float(len(numbers) - 1)
    return math.sqrt(variance)
  
def MeanAndStdDev(mydata):
    info = [(mean(attribute), std_dev(attribute)) for attribute in zip(*mydata)]
    # eg: list = [ [a, b, c], [m, n, o], [x, y, z]]
    # here mean of 1st attribute =(a + m+x), mean of 2nd attribute = (b + n+y)/3
    # delete summaries of last class
    del info[-1]
    return info
  
# find Mean and Standard Deviation under each class
def MeanAndStdDevForClass(mydata):
    info = {}
    dict = groupUnderClass(mydata)
    for classValue, instances in dict.items():
        info[classValue] = MeanAndStdDev(instances)
    return info
  
  
# Calculate Gaussian Probability Density Function
def calculateGaussianProbability(x, mean, stdev):
    expo = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
    return (1 / (math.sqrt(2 * math.pi) * stdev)) * expo
  
  
# Calculate Class Probabilities
def calculateClassProbabilities(info, test):
    probabilities = {}
    for classValue, classSummaries in info.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, std_dev = classSummaries[i]
            x = test[i]
            probabilities[classValue] *= calculateGaussianProbability(x, mean, std_dev)
    return probabilities
  
  
# Make prediction - highest probability is the prediction
def predict(info, test):
    probabilities = calculateClassProbabilities(info, test)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel
  
  
# returns predictions for a set of examples
def getPredictions(info, test):
    predictions = []
    for i in range(len(test)):
        result = predict(info, test[i])
        predictions.append(result)
    return predictions
  
# Accuracy score
def accuracy_rate(test, predictions):
    correct = 0
    for i in range(len(test)):
        if test[i][-1] == predictions[i]:
            correct += 1
    return (correct / float(len(test))) * 100.0
  
  
# driver code
  
# add the data path in your system
filename = r'data1.csv'
  
  
# load the file and store it in mydata list
mydata = csv.reader(open(filename, "rt"))
mydata = list(mydata)
mydata = encode_class(mydata)
for i in range(len(mydata)):
    mydata[i] = [float(x) for x in mydata[i]]
  
      
# split ratio = 0.7 
# 70% of data is training data and 30% is test data used for testing
ratio = 0.7
train_data, test_data = splitting(mydata, ratio)
print('Total number of examples are: ', len(mydata))
print('Out of these, training examples are: ', len(train_data))
print("Test examples are: ", len(test_data))
  
# prepare model
info = MeanAndStdDevForClass(train_data)

Total number of examples are:  399
Out of these, training examples are:  279
Test examples are:  120


In [ ]:
#Train data
df_train = pd.DataFrame(train_data)
print(df_train.shape)
df_train.head()

(279, 33)


,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,100140.0,-999.000,64.998,58.011,1.974,-999.0,-999.0,-999.0,2.165,1.974,...,0.0,-999.000,-999.000,-999.000,-999.0,-999.0,-999.0,0.000,4.597495,1.0
1,100185.0,-999.000,100.244,22.819,35.492,-999.0,-999.0,-999.0,0.635,0.975,...,1.0,36.039,-1.230,-0.716,-999.0,-999.0,-999.0,36.039,4.955874,1.0
2,100131.0,-999.000,85.551,51.971,17.796,-999.0,-999.0,-999.0,1.963,18.889,...,1.0,32.519,1.142,-2.585,-999.0,-999.0,-999.0,32.519,2.378133,1.0
3,100213.0,193.484,47.934,155.433,18.153,-999.0,-999.0,-999.0,4.007,18.153,...,0.0,-999.000,-999.000,-999.000,-999.0,-999.0,-999.0,0.000,6.967195,1.0
4,100161.0,118.303,60.852,88.652,29.310,-999.0,-999.0,-999.0,2.726,4.646,...,1.0,32.643,-1.420,-2.178,-999.0,-999.0,-999.0,32.643,2.325369,1.0


In [ ]:
#test data
df_test = pd.DataFrame(test_data)
print(df_test.shape)
df_test.head()

(120, 33)


,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,100004.0,175.864,16.915,134.805,16.405,-999.0,-999.0,-999.0,3.891,16.405,...,0.0,-999.000,-999.000,-999.000,-999.0,-999.0,-999.0,0.000,6.245333,1.0
1,100009.0,128.053,88.941,69.272,193.392,-999.0,-999.0,-999.0,1.609,28.859,...,1.0,167.735,-2.767,-2.514,-999.0,-999.0,-999.0,167.735,0.001502,0.0
2,100014.0,-999.000,109.412,14.398,17.323,-999.0,-999.0,-999.0,0.472,17.323,...,0.0,-999.000,-999.000,-999.000,-999.0,-999.0,-999.0,0.000,2.151199,1.0
3,100017.0,127.861,50.953,77.267,26.967,-999.0,-999.0,-999.0,2.833,26.967,...,0.0,-999.000,-999.000,-999.000,-999.0,-999.0,-999.0,0.000,0.001503,0.0
4,100021.0,90.736,18.674,60.231,25.156,-999.0,-999.0,-999.0,2.363,25.156,...,0.0,-999.000,-999.000,-999.000,-999.0,-999.0,-999.0,0.000,1.681611,1.0


In [ ]:
# test model
predictions = getPredictions(info, test_data)
accuracy = accuracy_rate(test_data, predictions)
print("Accuracy of your model is: ", accuracy)

Accuracy of your model is:  87.5
